In [1]:
import detect
import shutil
import cv2
import numpy as np
import os
import pymongo

C:\Users\gbuln\anaconda3\envs\luna\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['Yeji']
#collection3 = db.create_collection('Resident')


In [14]:
# 정기권 차량1등록
document1 = {
    'carnum': '02나4287',
    'registered': True,
    'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\02나4287_in_473947e7-3540-4169-98fa-7f7743ae3ed1.jpg'
}
collection3.insert_one(document1)

In [3]:
#정기권차량2등록
document2 = {
    'carnum': '01누0599',
    'registered': True,
    'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg'
}
collection3.insert_one(document2)

NameError: name 'collection3' is not defined

In [4]:
#입출차기록 db에 삽입
collection = db.create_collection('InAndOut')

CollectionInvalid: collection InAndOut already exists

In [6]:
# 위조차량 등장()

document3 = {
    'accessid' : 6,
    'intime': '2023-09-27 22:22:22',
    'outtime' : '',
    'carnum': '01누0599',
    'faked': False,
    'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01버0399(01누-0599).jpg'
}
#입출차기록에 넣어준다.
collection = db["InAndOut"]
collection.insert_one(document3)

In [7]:
import pymongo

# Connect to the MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['Yeji']
# Get the Resident collection
collection = db["Resident"]

# Find the document with carnum "01누0599" 01누0599인 차량을 찾는다.
cursor = collection.find({"carnum": document3['carnum']})

# Print the imageURL field of the document
for document in cursor:
    print(document["imageURL"])
    original_path = document["imageURL"] #원본 사진의 url 주소


C:\Users\gbuln\Downloads\fakenumber\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg


In [8]:
# source_path=original_path

In [9]:
save_path = f"C:\\Users\\gbuln\\Downloads\\{document3['carnum']}" #차량번호를 이름으로 한 폴더를 만들어준다.
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [10]:


#  헤드라이트만 크롭하기 위해 입출차내역 사진부터 옮겨주고,
src = document3["imageURL"]

# The destination directory for the copy
dst = save_path

# Copy the file
shutil.copy(src, dst)


'C:\\Users\\gbuln\\Downloads\\01누0599\\01버0399(01누-0599).jpg'

In [11]:

# 헤드라이트 크롭을 위해 원본의 사진도 옮겨준다(나중에 미리 크롭해뒀다가 db에 저장해두는게 빠를지도?)
src = original_path

# The destination directory for the copy
dst = save_path

# Copy the file
shutil.copy(src, dst)

'C:\\Users\\gbuln\\Downloads\\01누0599\\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg'

In [12]:
#detect 하는 코드
detect.run(weights=".\\headlights_v5.pt", 
                source=save_path,
                conf_thres=0.7,
                save_crop=True,
                save_txt=True,
                save_conf=True)

YOLOv5  v6.1-306-gfbe67e4 Python-3.9.15 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
image 1/2 C:\Users\gbuln\Downloads\01누0599\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg: 480x640 1 headlight, Done. (0.165s)
image 2/2 C:\Users\gbuln\Downloads\01누0599\01버0399(01누-0599).jpg: 480x640 1 headlight, Done. (0.156s)
Speed: 1.0ms pre-process, 160.3ms inference, 0.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp25
2 labels saved to runs\detect\exp25\labels


In [26]:
# 디렉토리에 있는 jpg 파일명을 한꺼번에 리스트에 추가해 주는 코드 
# 파일 이름 확인용으로 그냥 짰다.

root_path = '.\\runs\\detect\\exp24\\crops\\headlight\\'

In [16]:
# get a list of all files in the directory
files = os.listdir(root_path)

# create an empty list to store filenames
jpg_filenames = []

# iterate over the files
for file in files:
  # check if the file is a JPG file
  if file.endswith('.jpg'):
    # remove the '.jpg' from the filename
    filename = file[:-4]
    # add the filename to the list
    jpg_filenames.append(filename)

# print the filenames
print(jpg_filenames)

['01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400', '01버0399(01누-0599)']


In [17]:
# 한글 -> 영어로 변경 
import os
from hangul_romanize import Transliter
from hangul_romanize.rule import academic

for file in files:
    
    print(file)
    transliter = Transliter(academic)
    trans = transliter.translit(file)
    print(trans)
    os.rename(root_path+file, root_path+trans)

01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg
01nu0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg
01버0399(01누-0599).jpg
01beo0399(01nu-0599).jpg


In [24]:
import argparse
import cv2
import matplotlib.cm as cm
import torch
from pathlib import Path
from models.matching import Matching
from models.utils import (AverageTimer, VideoStreamer, make_matching_plot_fast, frame2tensor)

torch.set_grad_enabled(False)

def superglue(input, output_dir=None, image_glob=['*.png', '*.jpg', '*.jpeg'], 
              skip=1, max_length=1000000, resize=[640, 480], 
              superglue='indoor', max_keypoints=-1, keypoint_threshold=0.005,
              nms_radius=4, sinkhorn_iterations=20, match_threshold=0.2,
              show_keypoints=False, no_display=False, force_cpu=False):
    if len(resize) == 2 and resize[1] == -1:
        resize = resize[0:1]
    if len(resize) == 2:
        print('Will resize to {}x{} (WxH)'.format(
            resize[0], resize[1]))
    elif len(resize) == 1 and resize[0] > 0:
        print('Will resize max dimension to {}'.format(resize[0]))
    elif len(resize) == 1:
        print('Will not resize images')
    else:
        raise ValueError('Cannot specify more than two integers for --resize')

    device = 'cuda' if torch.cuda.is_available() and not force_cpu else 'cpu'
    print('Running inference on device \"{}\"'.format(device))
    config = {
        'superpoint': {
            'nms_radius': nms_radius,
            'keypoint_threshold': keypoint_threshold,
            'max_keypoints': max_keypoints
        },
        'superglue': {
            'weights': superglue,
            'sinkhorn_iterations': sinkhorn_iterations,
            'match_threshold': match_threshold,
        }
    }
    matching = Matching(config).eval().to(device)
    keys = ['keypoints', 'scores', 'descriptors']

    vs = VideoStreamer(input, resize, skip,
                       image_glob, max_length)
    frame, ret = vs.next_frame()
    assert ret, 'Error when reading the first frame (try different --input?)'

    frame_tensor = frame2tensor(frame, device)
    last_data = matching.superpoint({'image': frame_tensor})
    last_data = {k+'0': last_data[k] for k in keys}
    last_data['image0'] = frame_tensor
    last_frame = frame
    last_image_id = 0

    if output_dir is not None:
        print('==> Will write outputs to {}'.format(output_dir))
        Path(output_dir).mkdir(exist_ok=True)

    # Create a window to display the demo.
    if not no_display:
        cv2.namedWindow('SuperGlue matches', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('SuperGlue matches', 640*2, 480)
    else:
        print('Skipping visualization, will not show a GUI.')

    # Print the keyboard help menu.
    print('==> Keyboard control:\n'
          '\tn: select the current frame as the anchor\n'
          '\te/r: increase/decrease the keypoint confidence threshold\n'
          '\td/f: increase/decrease the match filtering threshold\n'
          '\tk: toggle the visualization of keypoints\n'
          '\tq: quit')

    timer = AverageTimer()

    while True:
        frame, ret = vs.next_frame()
        if not ret:
            print('Finished superglue function')
            break
        timer.update('data')
        stem0, stem1 = last_image_id, vs.i - 1

        frame_tensor = frame2tensor(frame, device)
        pred = matching({**last_data, 'image1': frame_tensor})
        kpts0 = last_data['keypoints0'][0].cpu().numpy()
        kpts1 = pred['keypoints1'][0].cpu().numpy()
        matches = pred['matches0'][0].cpu().numpy()
        confidence = pred['matching_scores0'][0].cpu().numpy()
        timer.update('forward')

        valid = matches > -1
        mkpts0 = kpts0[valid]
        mkpts1 = kpts1[matches[valid]]
        color = cm.jet(confidence[valid])
        global points
        text = [
            'SuperGlue',
            'Keypoints: {}:{}'.format(len(kpts0), len(kpts1)),
            'Matches: {}'.format(len(mkpts0))
        ]
        # Open the file in write mode
        with open(output_dir + '/matches.txt', 'w') as f:
            # Write the string to the file
            f.write(str(len(mkpts0)))
        points = len(mkpts0)
        print(len(mkpts0))

        k_thresh = matching.superpoint.config['keypoint_threshold']
        m_thresh = matching.superglue.config['match_threshold']
        small_text = [
            'Keypoint Threshold: {:.4f}'.format(k_thresh),
            'Match Threshold: {:.2f}'.format(m_thresh),
            'Image Pair: {:06}:{:06}'.format(stem0, stem1),
        ]
        out = make_matching_plot_fast(
            last_frame, frame, kpts0, kpts1, mkpts0, mkpts1, color, text,
            path=None, show_keypoints=show_keypoints, small_text=small_text)

        if not no_display:
            cv2.imshow('SuperGlue matches', out)
            key = chr(cv2.waitKey(1) & 0xFF)
            if key == 'q':
                vs.cleanup()
                print('Exiting (via q) demo_superglue.py')
                break
            elif key == 'n':  # set the current frame as anchor
                last_data = {k+'0': pred[k+'1'] for k in keys}
                last_data['image0'] = frame_tensor
                last_frame = frame
                last_image_id = (vs.i - 1)
            elif key in ['e', 'r']:
                # Increase/decrease keypoint threshold by 10% each keypress.
                d = 0.1 * (-1 if key == 'e' else 1)
                matching.superpoint.config['keypoint_threshold'] = min(max(
                    0.0001, matching.superpoint.config['keypoint_threshold']*(1+d)), 1)
                print('\nChanged the keypoint threshold to {:.4f}'.format(
                    matching.superpoint.config['keypoint_threshold']))
            elif key in ['d', 'f']:
                # Increase/decrease match threshold by 0.05 each keypress.
                d = 0.05 * (-1 if key == 'd' else 1)
                matching.superglue.config['match_threshold'] = min(max(
                    0.05, matching.superglue.config['match_threshold']+d), .95)
                print('\nChanged the match threshold to {:.2f}'.format(
                    matching.superglue.config['match_threshold']))
            elif key == 'k':
                show_keypoints = not show_keypoints

        timer.update('viz')
        timer.print()

        if output_dir is not None:
            #stem = 'matches_{:06}_{:06}'.format(last_image_id, vs.i-1)
            stem = 'matches_{:06}_{:06}'.format(stem0, stem1)
            out_file = str(Path(output_dir, stem + '.png'))
            print('\nWriting image to {}'.format(out_file))
            cv2.imwrite(out_file, out)
            
    vs.cleanup()
    return points

In [27]:
a = superglue(root_path, output_dir=save_path, image_glob=['*.png', '*.jpg', '*.jpeg'], 
              skip=1, max_length=1000000, resize=[320, 240], 
              superglue='indoor', max_keypoints=2048, keypoint_threshold=0.1,
              nms_radius=4, sinkhorn_iterations=20, match_threshold=0.3,
              show_keypoints=False, no_display=True, force_cpu=False)

print(a)

Will resize to 320x240 (WxH)
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
==> Processing image directory input: .\runs\detect\exp24\crops\headlight\
==> Will write outputs to C:\Users\gbuln\Downloads\01누0599
Skipping visualization, will not show a GUI.
==> Keyboard control:
	n: select the current frame as the anchor
	e/r: increase/decrease the keypoint confidence threshold
	d/f: increase/decrease the match filtering threshold
	k: toggle the visualization of keypoints
	q: quit
19
[Timer] data=0.005 forward=0.201 viz=0.017 total=0.223 sec {4.5 FPS} 
Writing image to C:\Users\gbuln\Downloads\01누0599\matches_000000_000001.png
Finished demo_superglue.py
19


In [28]:
collection = db["InAndOut"]
if a < 20:
    collection.update_one({"accessid": 5}, {"$set": {"faked": True}})
    print("faked")
else:
    pass

faked


In [22]:
cursor = collection.find({})
for document in cursor:
    print(document)

{'_id': ObjectId('63bc08a1ce004124447a391e'), 'accessid': 1, 'intime': '2023-09-27 22:22:22', 'outtime': '', 'carnum': '01누0599', 'faked': True, 'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01버0399(01누-0599).jpg'}
{'_id': ObjectId('63bcbc8b8d9eef5f7166dca0'), 'accessid': 2, 'intime': '2023-09-27 23:22:22', 'outtime': '', 'carnum': '01누0599', 'faked': False, 'imageURL': '01누0599_in_9d6eda83-f0aa-4d55-9595-34e1ff43f4c1.jpg'}
{'_id': ObjectId('63bcbd728d9eef5f7166dca3'), 'accessid': 3, 'intime': '2023-09-27 23:22:22', 'outtime': '', 'carnum': '01누0599', 'faked': False, 'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01누0599_in_9d6eda83-f0aa-4d55-9595-34e1ff43f4c1.jpg'}
{'_id': ObjectId('63bd034fafe5b30dded99a92'), 'accessid': 5, 'intime': '2023-09-27 22:22:22', 'outtime': '', 'carnum': '01누0599', 'faked': True, 'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01버0399(01누-0599).jpg'}
{'_id': ObjectId('63bd189436895d8b7c26488d'), 'accessid': 5, 'intime': '2023-09-27 22:2

In [23]:
shutil.rmtree(root_path)
shutil.rmtree(save_path)